In [2]:
#--------------------MSE Calculation--------------------#

print('Running MSE Calculation')
import numpy as np
import pandas as pd
from sklearn.utils import resample

cln_data_path = '../output/Data Background/'
bysess_all_1s = pd.read_excel(cln_data_path + 'bysess_all_1s_first.xlsx')
bysess_all_2s = pd.read_excel(cln_data_path + 'bysess_all_2s_first.xlsx')
actual_pi_1s = bysess_all_1s.mean(axis=0).values
actual_pi_2s = bysess_all_2s.mean(axis=0).values
actual_inf_1s = 0.5 * (1 - actual_pi_1s[6] + actual_pi_1s[4])
actual_inf_2s = 0.5 * (1 - actual_pi_2s[9] + actual_pi_2s[7])

eqa = np.load('../output/Theory Section/eqa.npz')
lmix_pi_1s, pool_pi_1s, lmix_pi_2s, hmix_pi_2s, pool_pi_2s = (eqa[key] for key in ['lmix_pi_1s', 'pool_pi_1s', 'lmix_pi_2s', 'hmix_pi_2s', 'pool_pi_2s'])
pool_pi_1s[0] = min(1/3, actual_pi_1s[0])
pool_pi_1s[1] = 1 - min(1/3, actual_pi_1s[0])
pool_pi_2s[0] = actual_pi_2s[0]
pool_pi_2s[1] = 1 - actual_pi_2s[0]
theory_pi_1s = [lmix_pi_1s, pool_pi_1s]
theory_pi_2s = [lmix_pi_2s, hmix_pi_2s, pool_pi_2s]
inf = np.load('../output/Theory Section/inf.npz')

# Actual MSE
MSE_1s = []
for pi in theory_pi_1s:
    mse = np.mean((actual_pi_1s - pi)**2)
    MSE_1s.append(mse)
MSE_2s = []
for pi in theory_pi_2s:
    mse = np.mean((actual_pi_2s - pi)**2)
    MSE_2s.append(mse)

# Bootstrapping
actual_pis_1s = bysess_all_1s.values
actual_pis_2s = bysess_all_2s.values
n_bootstrap = 10000
n_data_points = len(actual_pis_1s)

MSEs_1s = []
MSEs_2s = []
for i in range(n_bootstrap):
    resampled_data_1s = resample(actual_pis_1s, replace=True, n_samples=n_data_points)
    resampled_data_2s = resample(actual_pis_2s, replace=True, n_samples=n_data_points)
    bootstrap_sample_1s = np.mean(resampled_data_1s, axis=0)
    bootstrap_sample_2s = np.mean(resampled_data_2s, axis=0)
    MSEs_1s.append([np.mean((bootstrap_sample_1s - pi)**2) for pi in theory_pi_1s])
    MSEs_2s.append([np.mean((bootstrap_sample_2s - pi)**2) for pi in theory_pi_2s])

SE_1s = np.var(MSEs_1s, axis=0)**0.5
SE_2s = np.var(MSEs_2s, axis=0)**0.5

selection_1s = [mse[0] < mse[1] for mse in MSEs_1s]
p_lmix_1s = np.mean(selection_1s)

selection_lmix_2s = [mse[0] < min(mse[1], mse[2]) for mse in MSEs_2s]
selection_hmix_2s = [mse[1] < min(mse[0], mse[2]) for mse in MSEs_2s]
p_lmix_2s = np.mean(selection_lmix_2s)
p_hmix_2s = np.mean(selection_hmix_2s)

# Create table
tex_code = r"""
\begin{table}[htbp]
\renewcommand{\arraystretch}{1}
\begin{tabular}{lccc}
%\multicolumn{4}{c}{\textbf{Table 1: Nash Equilibrium Selection using MSE}} \\
\toprule
\multicolumn{4}{c}{$n = 1$ firm} \\
\midrule 
& \textbf{Low-types mix} &  & \textbf{Pooling} \\
\cmidrule(lr){2-2} \cmidrule(lr){4-4} 
"""

row1 = "MSE & " + format(MSE_1s[0], ".3f") + " (" + format(SE_1s[0], ".3f") + ")" + " & " + " & " + format(MSE_1s[1], ".3f") + " (" + format(SE_1s[1], ".3f") + ")" + " \\\\"
row2 = "likelihood & " + format(p_lmix_1s, ".3f") + " & " + " & " + format(1 - p_lmix_1s, ".3f") + " \\\\"
row3 = "$\mathbb{P}(p_H|v_H)$ & " + format(lmix_pi_1s[4], ".3f") + " & " + " & " + format(pool_pi_1s[4], ".3f") + " \\\\"
row4 = "$\mathbb{P}(p_H|v_L)$ & " + format(lmix_pi_1s[6], ".3f") + " & " + " & " + format(pool_pi_1s[6], ".3f") + " \\\\"
row5 = "informativeness & " + format(inf['inf_lmix_1s'], ".3f") + " & " + " & " + format(inf['inf_pool_1s'], ".3f") + " \\\\"

tex_code += row1 + "\n" + row2 + "\n" + row3 + "\n" + row4 + "\n" + row5

tex_code += r"""
\midrule
\multicolumn{4}{c}{$n = 2$ firms} \\
\midrule 
& \textbf{Low-types mix} & \textbf{High-types mix} & \textbf{Pooling} \\
\cmidrule(lr){2-2} \cmidrule(lr){3-3} \cmidrule(lr){4-4}
"""

row6  = "MSE & " + format(MSE_2s[0], ".3f") + " (" + format(SE_2s[0], ".3f") + ")" + " & " + format(MSE_2s[1], ".3f") + " (" + format(SE_2s[1], ".3f") + ")" + " & " + format(MSE_2s[2], ".3f") + " (" + format(SE_2s[2], ".3f") + ")" + " \\\\"
row7  = "likelihood & " + format(p_lmix_2s, ".3f") + " & " + format(p_hmix_2s, ".3f") + " & " + format(1 - p_lmix_2s - p_hmix_2s, ".3f") + " \\\\"
row8  = "$\mathbb{P}(p_H|v_H)$ & " + format(lmix_pi_2s[7], ".3f") + " & " + format(hmix_pi_2s[7], ".3f") + " & " + format(pool_pi_2s[7], ".3f") + " \\\\"
row9  = "$\mathbb{P}(p_H|v_L)$ & " + format(lmix_pi_2s[9], ".3f") + " & " + format(hmix_pi_2s[9], ".3f") + " & " + format(pool_pi_2s[9], ".3f") + " \\\\"
row10 = "informativeness & " + format(inf['inf_lmix_2s'], ".3f") + " & " + format(inf['inf_hmix_2s'], ".3f") + " & " + format(inf['inf_pool_2s'], ".3f") + " \\\\"

tex_code += row6 + "\n" + row7 + "\n" + row8 + "\n" + row9 + "\n" + row10

tex_code += r"""
\bottomrule
\end{tabular}
\caption{Nash Equilibrium Selection using MSE}
\label{mse_table}
\end{table}
"""

with open("../output/MSE Calculation/mse_table.tex", "w") as file:
    file.write(tex_code)
print('Created mse_table.tex')

mse_sep_likelihood_1s  = 100 * p_lmix_1s
mse_pool_likelihood_2s = 100 * (1 - p_lmix_2s - p_hmix_2s)

with open("../output/MSE Calculation/mse_vars.tex", "w") as f:
    f.write(f"\\newcommand{{\\msesepone}}{{{mse_sep_likelihood_1s:.0f}}}\n")
    f.write(f"\\newcommand{{\\msepooltwo}}{{{mse_pool_likelihood_2s:.0f}}}\n")
print('Created mse_vars.tex')
print('\n')


Running MSE Calculation
Created mse_table.tex
Created mse_vars.tex


